# Nationalratswahlen

In [3]:
!pip install pyaxis

     |████████████████████████████████| 15.7 MB 6.8 MB/s eta 0:00:01
     |████████████████████████████████| 11.5 MB 9.6 MB/s eta 0:00:01
     |████████████████████████████████| 794 kB 61.1 MB/s eta 0:00:01
  Created wheel for pyaxis: filename=pyaxis-0.3.4-py3-none-any.whl size=10766 sha256=4cdb0200ac10f4323b8330c6f92ed3c3054c82c548e070d67471f6885fe58492
  Stored in directory: /home/jovyan/.cache/pip/wheels/62/9d/72/93ff83bda641a1843128911c9a94a7f739aaf4788388f5db67
  Created wheel for pyjstat: filename=pyjstat-2.2.1-py3-none-any.whl size=19429 sha256=ed0e4e8a15fd7b7b8915ae523e9ba6f058ccf5a4f26b7aa76b8bb43fa681aa18
  Stored in directory: /home/jovyan/.cache/pip/wheels/94/28/15/0c98e3d81093d35fb6bd1f28295e145788970d18b7a0076f22
Successfully built pyaxis pyjstat
You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.


In [4]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

In [5]:
from pyaxis import pyaxis
import pandas as pd
import sys

In [12]:
# load data from bfs
URL = 'https://www.bfs.admin.ch/bfsstatic/dam/assets/12948000/master'

In [13]:
px = pyaxis.parse(URL, encoding='ISO-8859-2')

In [23]:
df = px['DATA']
print(df.shape)
df.head()

(570024, 4)


,Kanton (-) / Bezirk (>>) / Gemeinde (......),Jahr,Ergebnisse,DATA
0,Schweiz,2019,Wahlberechtigte,5459218
1,Schweiz,2019,Wählende / Eingelegte Wahlzettel (WZ),2462641
2,Schweiz,2019,Wahlbeteiligung [%],45.1097758
3,Schweiz,2019,Leere WZ,9366
4,Schweiz,2019,Ungültige WZ,29015


In [24]:
# rename 'Kanton (-) / Bezirk (>>) / Gemeinde (......)' to 'name_de'
df = df.rename(columns={
    'Kanton (-) / Bezirk (>>) / Gemeinde (......)': 'name_de', 
    'Jahr': 'year', 
    'Ergebnisse': 'results', 
    'DATA': 'data'
})
df.head()

,name_de,year,results,data
0,Schweiz,2019,Wahlberechtigte,5459218
1,Schweiz,2019,Wählende / Eingelegte Wahlzettel (WZ),2462641
2,Schweiz,2019,Wahlbeteiligung [%],45.1097758
3,Schweiz,2019,Leere WZ,9366
4,Schweiz,2019,Ungültige WZ,29015


In [27]:
df['name_de'] = df['name_de'].str.replace('.', '')
df['name_de'] = df['name_de'].str.replace('>> ', '')
df['name_de'] = df['name_de'].str.replace('   ', '')

/tmp/ipykernel_823/2170717606.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['name_de'] = df['name_de'].str.replace('.', '')


In [31]:
set(df['results'])

{'Gültige WZ',
 'Kandidatenstimmen aus unveränderten WZ',
 'Kandidatenstimmen aus veränderten WZ',
 'Leere WZ',
 'Listenstimmen aus unveränderten WZ',
 'Listenstimmen aus veränderten WZ',
 'Total Kandidatenstimmen',
 'Total Listenstimmen',
 'Total Zusatzstimmen',
 'Ungültige WZ',
 'Unveränderte WZ',
 'Veränderte WZ',
 'WZ ohne Listenbezeichnung',
 'Wahlberechtigte',
 'Wahlbeteiligung [%]',
 'Wählende / Eingelegte Wahlzettel (WZ)',
 'Zusatzstimmen aus unveränderten WZ',
 'Zusatzstimmen aus veränderten WZ'}

In [33]:
metadata= {}
for col in ['name_de', 'results']:
    conversion_dict = {}
    options = set(df[col])
    for i, option in enumerate(options):
        conversion_dict[option] = i
    df[col] = df[col].map(conversion_dict)
    metadata[col] = conversion_dict

In [34]:
df.head()

,name_de,year,results,data
0,459,2019,11,5459218
1,459,2019,13,2462641
2,459,2019,7,45.1097758
3,459,2019,8,9366
4,459,2019,14,29015


In [39]:
# create new dataframe
df = pd.DataFrame({
    'indicator_id': 'TBD',
    'spatialunit_id': df['name_de'],
    'time_value': df['year'],
    'time_info_id': 1,
    'results': df['results'],
    'value': df['data']
})

In [40]:
df.head()

,indicator_id,spatialunit_id,time_value,time_info_id,results,value
0,TBD,459,2019,1,11,5459218
1,TBD,459,2019,1,13,2462641
2,TBD,459,2019,1,7,45.1097758
3,TBD,459,2019,1,8,9366
4,TBD,459,2019,1,14,29015


In [19]:
# export as csv
df.to_csv('national_council_elections.csv', index=False, na_rep='NA')